In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/glovetwitter27b/glove.twitter.27B.200d.txt
/kaggle/input/glovetwitter27b/glove.twitter.27B.25d.txt
/kaggle/input/glovetwitter27b/glove.twitter.27B.50d.txt
/kaggle/input/glovetwitter27b/glove.twitter.27B.100d.txt
/kaggle/input/disaster-tweets-text-embeddings/test_text_embeddings.csv
/kaggle/input/disaster-tweets-text-embeddings/train_text_embeddings.csv


In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.2 MB/s eta 0:00:00


In [3]:
# # Set up and test the OpenAI API
# import openai
# openai.api_key = ""

# # create a chat completion
# chat_completion = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {
#             "role": "user",
#             "content": "Output a single letter, 1 if the following tweet is about a real disaster and 0 if not:\
#             Just happened a terrible car crash"
#         }])

# # print the chat completion
# print(chat_completion.choices[0].message.content)

In [4]:
import nltk
nltk.data.path.append('/kaggle/working')
nltk.download("wordnet", download_dir='/kaggle/working')

import os
os.environ['NLTK_DATA'] = '/kaggle/working'

!mkdir /kaggle/working/corpora/wordnet
!unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora

[nltk_data] Downloading package wordnet to /kaggle/working...
Archive:  /kaggle/working/corpora/wordnet.zip
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpora/wordnet/noun.exc  
  inflating: /kaggle/working/corpora/wordnet/verb.exc  
  inflating: /kaggle/working/corpora/wordnet/README  
  inflating: /kaggle/working/corpora/wordnet/index.sense  
  inflating: /kaggle/working/corpora/wordnet/data.noun  
  inflating: /kaggle/working/corpora/wor

In [5]:
train_file_path = "/kaggle/input/nlp-getting-started/train.csv"
train_data = pd.read_csv(train_file_path)

print(train_data.columns)

test_file_path = "/kaggle/input/nlp-getting-started/test.csv"
test_data = pd.read_csv(test_file_path)

print(test_data.columns)

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
Index(['id', 'keyword', 'location', 'text'], dtype='object')


In [6]:
import re
import os
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from openai.embeddings_utils import get_embedding

# Class responsible for handling the input data
class Dataset:
    learn_len = None
    
    combined_data = None
    train_data = None
    test_data = None
    learn_data = None
    assess_data = None
    
    target_data = None
    
    combined_encoded_text = None
    train_encoded_text = None
    test_encoded_text = None
    learn_encoded_text = None
    assess_encoded_text = None
    
    combined_sequences = None
    train_sequences = None
    test_sequences = None
    learn_sequences = None
    assess_sequences = None
    
    vocab_size = None
    tokenizer = None
    max_sequence_length = 28
    
    train_embeddings_read_file_path = "/kaggle/input/disaster-tweets-text-embeddings/train_text_embeddings.csv"
    test_embeddings_read_file_path = "/kaggle/input/disaster-tweets-text-embeddings/test_text_embeddings.csv"
    train_embeddings_file_path = "/kaggle/working/train_text_embeddings.csv"
    test_embeddings_file_path = "/kaggle/working/test_text_embeddings.csv"
    combined_text_embeddings = None
    train_text_embeddings = None
    test_text_embeddings = None
    learn_text_embeddings = None
    assess_text_embeddings = None

    # data is pandas DataFrame
    def __init__(self, train_data, test_data, learn_ratio, assess_ratio):
        assert learn_ratio + assess_ratio == 1, \
        "The sum of learn_ratio and assess_ratio should be equal to 1"
        
        self.train_data = train_data
        self.test_data = test_data
        self.learn_len = int(learn_ratio * len(self.train_data))
        self.learn_data  = self.train_data.iloc[:self.learn_len]
        self.assess_data = self.train_data.iloc[self.learn_len:]
        self.combined_data = pd.concat([self.train_data.iloc[:, :-1], self.test_data])
        self.target_data = self.train_data.iloc[:, -1]
        
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.combined_data['text'])
        self.vocab_size = len(self.tokenizer.word_index) + 1

        self.create_sequences()
        self.create_encoded_texts()
        self.create_text_embeddings()
        
    def create_sequences(self):
        self.combined_sequences = pad_sequences(
            self.tokenizer.texts_to_sequences(self.combined_data['text']), 
            maxlen=self.max_sequence_length, 
            padding='post')
        self.train_sequences = pad_sequences(
            self.tokenizer.texts_to_sequences(self.train_data['text']),
            maxlen=self.max_sequence_length,
            padding='post')
        self.test_sequences = pad_sequences(
            self.tokenizer.texts_to_sequences(self.test_data['text']),
            maxlen=self.max_sequence_length,
            padding='post')
        self.learn_sequences = pad_sequences(
            self.tokenizer.texts_to_sequences(self.learn_data['text']),
            maxlen=self.max_sequence_length,
            padding='post')
        self.assess_sequences = pad_sequences(
            self.tokenizer.texts_to_sequences(self.assess_data['text']),
            maxlen=self.max_sequence_length,
            padding='post')
        
    def create_encoded_texts(self):
        encoded_documents = self.tfidf_encode_documents(self.combined_data['text'])
        self.combined_encoded_text = pd.DataFrame.sparse.from_spmatrix(encoded_documents)
        self.train_encoded_text = self.combined_encoded_text.iloc[:len(self.train_data)]
        self.test_encoded_text = self.combined_encoded_text.iloc[len(self.train_data):]
        self.learn_data_encoded_text = self.train_encoded_text[:self.learn_len]
        self.assess_data_encoded_text = self.train_encoded_text[self.learn_len:]
        
    def create_text_embeddings(self):
        if self.try_to_load_embeddings():
            print("Successfully loaded text embeddings from local storage !")
            return
        else:
            print("Failed to load text embeddings from local storage ... ")
            self.generate_embeddings()
            self.save_embeddings()
            
    def lemmatize(self, text):
        words = re.findall(r'\w+', text.lower())
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(word) for word in words]
    
    def tfidf_encode_documents(self, documents):
        vectorizer = TfidfVectorizer(tokenizer = self.lemmatize)
        return vectorizer.fit_transform(documents)
    
    def try_to_load_embeddings(self):
        if self.try_to_read_embeddings(self.train_embeddings_file_path, self.test_embeddings_file_path) or \
                self.try_to_read_embeddings(self.train_embeddings_read_file_path, self.test_embeddings_read_file_path):
            self.combined_text_embeddings = pd.concat([self.train_text_embeddings, self.test_text_embeddings])
            self.learn_text_embeddings = self.train_text_embeddings[:self.learn_len]
            self.assess_text_embeddings = self.train_text_embeddings[self.learn_len:]
            return True
        return False
    
    def try_to_read_embeddings(self, train_file_path, test_file_path):
        if os.path.exists(train_file_path) and os.path.exists(test_file_path):
            self.train_text_embeddings = pd.read_csv(train_file_path)
            self.test_text_embeddings = pd.read_csv(test_file_path)
            return True
        return False
    
    def generate_embeddings(self, model="text-embedding-ada-002"):
        print("Generating embeddings using OpenAI API ... ")
        self.combined_text_embeddings = pd.DataFrame(self.combined_data.text.apply(
            lambda x: get_embedding(x.replace("\n", " "), engine=model)).tolist())
        self.train_text_embeddings = self.combined_text_embeddings.iloc[:len(self.train_data)]
        self.test_text_embeddings = self.combined_text_embeddings.iloc[len(self.train_data):]
        self.learn_text_embeddings = self.train_text_embeddings.iloc[:self.learn_len]
        self.assess_text_embeddings = self.train_text_embeddings.iloc[self.learn_len:]
        print("Generated embeddings using OpenAI API ... ")

    def save_embeddings(self):
        print("Savings embeddings to local storage ... ")
        self.train_text_embeddings.to_csv(self.train_embeddings_file_path)
        self.test_text_embeddings.to_csv(self.test_embeddings_file_path)
        

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [7]:
dataset = Dataset(train_data, test_data, 0.8, 0.2)

assert len(dataset.learn_data) + len(dataset.assess_data) == len(dataset.train_data), \
"The size of the learn_data and assess_data should total to the size of data"

print("Learn dataset len={}".format(len(dataset.learn_data)))
print("Assess dataset len={}".format(len(dataset.assess_data)))
print("Total dataset len={}".format(len(dataset.train_data)))

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Successfully loaded text embeddings from local storage !
Learn dataset len=6090
Assess dataset len=1523
Total dataset len=7613


In [8]:
print(dataset.combined_text_embeddings.shape)
print(dataset.train_text_embeddings.shape)
print(dataset.test_text_embeddings.shape)
print(dataset.learn_text_embeddings.shape)
print(dataset.assess_text_embeddings.shape)

(10876, 1537)
(7613, 1537)
(3263, 1537)
(6090, 1537)
(1523, 1537)


In [9]:
# Neural Network model

import tensorflow as tf
from tensorflow import keras

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

input_size = dataset.train_encoded_text.shape[1]
hidden_layer_size = 10

tfidf_model = keras.models.Sequential()
tfidf_model.add(keras.layers.Dense(hidden_layer_size, activation='swish', input_shape=(input_size,)))
tfidf_model.add(keras.layers.Dense(2, activation='softmax'))
tfidf_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

tfidf_model.fit(dataset.train_encoded_text, dataset.target_data, epochs = 50)

Num GPUs Available:  1
Epoch 1/50
238/238 [==============================] - 5s 5ms/step - loss: 0.6365 - accuracy: 0.6534
Epoch 2/50
238/238 [==============================] - 1s 5ms/step - loss: 0.4642 - accuracy: 0.8386
Epoch 3/50
238/238 [==============================] - 1s 6ms/step - loss: 0.3312 - accuracy: 0.8869
Epoch 4/50
238/238 [==============================] - 1s 5ms/step - loss: 0.2448 - accuracy: 0.9222
Epoch 5/50
238/238 [==============================] - 1s 5ms/step - loss: 0.1839 - accuracy: 0.9438
Epoch 6/50
238/238 [==============================] - 1s 6ms/step - loss: 0.1386 - accuracy: 0.9573
Epoch 7/50
238/238 [==============================] - 2s 7ms/step - loss: 0.1059 - accuracy: 0.9715
Epoch 8/50
238/238 [==============================] - 1s 5ms/step - loss: 0.0817 - accuracy: 0.9798
Epoch 9/50
238/238 [==============================] - 1s 5ms/step - loss: 0.0640 - accuracy: 0.9867
Epoch 10/50
238/238 [==============================] - 1s 5ms/step - loss: 0.

In [10]:
import numpy as np
import collections

def load_glove_model(file_path, size, verbose = 1):
    model = collections.defaultdict(lambda: np.array([0.0 for _ in range(size)]))
    with open(file_path) as f:
        for line in f:
            tokens = line.split(' ')
            word = tokens[0]
            embeddings = np.array([float(value) for value in tokens[1:]])
            model[word] = embeddings
    if verbose >= 1:
        print("Words loaded!")
    return model

In [11]:
# Read the GloVe with pandas
import numpy as np

embedding_size = 100
glove_file_path = "/kaggle/input/glovetwitter27b/glove.twitter.27B.100d.txt"
glove_embeddings = load_glove_model(glove_file_path, embedding_size)

Words loaded!


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.initializers import Constant
import numpy as np

print(dataset.vocab_size)
embedding_matrix = np.zeros((dataset.vocab_size, embedding_size))
for word, i in dataset.tokenizer.word_index.items():
    embedding_matrix[i] = glove_embeddings[word]
    
embedding_model = Sequential()
embedding_layer = Embedding(
    dataset.vocab_size,
    embedding_size,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=dataset.max_sequence_length,
    trainable=False)
embedding_model.add(embedding_layer)
embedding_model.add(LSTM(100))
embedding_model.add(Dense(2, activation='softmax'))
embedding_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

embedding_model.fit(dataset.train_sequences, dataset.target_data, epochs = 50)

29320
Epoch 1/50
238/238 [==============================] - 4s 5ms/step - loss: 0.4766 - accuracy: 0.7863
Epoch 2/50
238/238 [==============================] - 1s 5ms/step - loss: 0.4245 - accuracy: 0.8118
Epoch 3/50
238/238 [==============================] - 1s 5ms/step - loss: 0.4095 - accuracy: 0.8224
Epoch 4/50
238/238 [==============================] - 1s 5ms/step - loss: 0.3936 - accuracy: 0.8354
Epoch 5/50
238/238 [==============================] - 1s 5ms/step - loss: 0.3749 - accuracy: 0.8421
Epoch 6/50
238/238 [==============================] - 1s 6ms/step - loss: 0.3586 - accuracy: 0.8500
Epoch 7/50
238/238 [==============================] - 1s 5ms/step - loss: 0.3386 - accuracy: 0.8592
Epoch 8/50
238/238 [==============================] - 1s 5ms/step - loss: 0.3251 - accuracy: 0.8681
Epoch 9/50
238/238 [==============================] - 1s 5ms/step - loss: 0.3017 - accuracy: 0.8782
Epoch 10/50
238/238 [==============================] - 1s 5ms/step - loss: 0.2835 - accuracy: 

In [13]:
# Neural Network model

import tensorflow as tf
from tensorflow import keras

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

print(dataset.test_text_embeddings)

input_size = dataset.train_text_embeddings.shape[1]
hidden_layer_size = 10

openai_model = keras.models.Sequential()
openai_model.add(keras.layers.Dense(hidden_layer_size, activation='swish', input_shape=(input_size,)))
openai_model.add(keras.layers.Dense(2, activation='softmax'))
openai_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

openai_model.fit(dataset.train_text_embeddings, dataset.target_data, epochs = 500)

Num GPUs Available:  1
      Unnamed: 0         0         1         2         3         4         5  \
0           7613 -0.021635  0.008594  0.009741  0.003018 -0.026378  0.011286   
1           7614 -0.002266 -0.001679  0.021356 -0.014359  0.002140  0.009852   
2           7615 -0.025502 -0.008412 -0.014390  0.005832 -0.020620  0.023935   
3           7616 -0.010744 -0.013325 -0.012183  0.004142 -0.024717  0.009615   
4           7617  0.003850 -0.018499  0.005596  0.017528 -0.012945  0.008153   
...          ...       ...       ...       ...       ...       ...       ...   
3258       10871 -0.005213 -0.010868 -0.000101 -0.029862 -0.000932 -0.002580   
3259       10872 -0.023784 -0.007963 -0.018151  0.011523 -0.007818  0.012971   
3260       10873 -0.009089 -0.008814  0.016513 -0.018191 -0.026394  0.014942   
3261       10874 -0.013321  0.005856 -0.016371 -0.018341 -0.008559  0.016996   
3262       10875 -0.007721 -0.030153 -0.022838 -0.016321 -0.016713  0.025137   

             6  

In [14]:
# output = np.array([0 for i in range(len(test_data))])

openai_output = openai_model.predict(dataset.test_text_embeddings)
embedding_output = embedding_model.predict(dataset.test_sequences)
tfidf_output = tfidf_model.predict(dataset.test_encoded_text)

output = embedding_output * tfidf_output * openai_output
answer = [0 if row[0] > row[1] else 1 for row in output]

predictions = pd.DataFrame({
    'id': test_data['id'],
    'target': answer
})

predictions.to_csv("submission.csv", index = False)

102/102 [==============================] - 0s 4ms/step


In [15]:
# Use OpenAPI Embeddings API to get the text embedding
print(len(dataset.test_data))
print(dataset.test_data.iloc[0])

3263
id                                           0
keyword                                    NaN
location                                   NaN
text        Just happened a terrible car crash
Name: 0, dtype: object
